In [ ]:
dane = {'blue_more': 0, 
        'sabres': 0, 
        'jokers': 0, 
        'gold': 3, 
        'extra_money_fivecards': 0, 
        'black_more': 0, 
        'points': 0, 
        'extra_card': 0, 
        'expeditions': 0, 
        'red_more': 0, 
        'anchors': 0, 
        'green_more': 0, 
        'crosses': 0, 
        'extra_money_notrade': 0, 
        'yellow_more': 0, 
        'houses': 0, 
        'pay_less': 0
       }

In [26]:
import random
import json
from IPython.display import Image, clear_output

In [27]:
class Settings(object):
    def __init__(self):
        settings_file = 'game_settings.ini'
        self.settings = self.__read_to_tuples__(settings_file)
        
    def __read_file__(self, filename):
        '''Function for read files to variables.'''
        with open(filename, 'r') as file_input:
            return json.load(file_input)
    
    def __tuple_inception__(self, data_set):
        '''Function for change type of read data sets as tuples.'''
        if type(data_set) == dict:
            for key, val in data_set.items():
                if type(val) == list:
                    data_set[key] = tuple(val)
        elif type(data_set) == list:
            for lst in data_set:
                for idx in range(len(lst)):
                    if type(lst[idx]) == list:
                        lst[idx] = tuple(lst[idx])
                data_set[data_set.index(lst)] = tuple(lst)
            data_set = tuple(data_set)
        return data_set
    
    def __read_to_tuples__(self, filename):
        '''Connection both above functions, better readable code.'''
        return self.__tuple_inception__(self.__read_file__(filename))

In [28]:
class Deck(object):
    def __init__(self):
        deck_file = 'pr_deck.txt'
        self.full_deck = Settings().__read_to_tuples__(deck_file)
        self.choosen = [] # catch choosen cards
        self.expeditions = [] # catch expeditions laying on table
        random.shuffle(self.choosen)

    def choose_card_from_deck(self):
        '''Function choosing card from full deck.'''
        self.choosing = random.choice(self.full_deck)
        if self.choosing not in (self.choosen, self.expeditions):
            if self.choosing[0] == 'expedition':
                self.expeditions.append(self.choosing) # add card to expeditions set
                return self.choose_card_from_deck()
            else:
                self.choosen.append(self.choosing) # add card to choosen set - bin
                return Card(self.choosing)
        else:
            return self.choose_card_from_deck()
        
    def reset_choosen_and_expeditions(self):
        '''Cleaning lists of choosen cards and cards of expeditions laying on table.'''
        self.choosen = []
        self.expeditions = []

In [29]:
class Card(object):
    def __init__(self, card_data):
        '''Set variables for choosen card.'''
        self.name = card_data[0][0]
        self.name_diff = card_data[0][Settings().settings['choosen_language']]
        self.color = card_data[1]
        self.points = card_data[2]
        self.cost = card_data[3]
        self.feature = card_data[4]
        self.nr = card_data[5]

In [30]:
class Player(object):
    def __init__(self, id):
        self.gid = id
        self.card = None
        self.hand = []
        player_data_file = 'data_dict.txt'
        card_description_file = 'card_description.ini'
        self.data_player = Settings().__read_to_tuples__(player_data_file) # catch data-statistics for player
        self.card_description = Settings().__read_to_tuples__(card_description_file) # catch card descriptions
        self.ch_lang = Settings().settings['choosen_language']
        
    def pickup_card(self, card):
        '''Picks up card from set of available cards.'''
        self.card = card
        
    def take_card(self):
        '''Take card - become owner of card.'''
        self.hand.append(self.card)
        
    def cards_in_hand(self):
        '''Return cards in hand of player.'''
        return self.hand
        
    def show_card(self):
        '''Shows card's data.'''
        return (self.card.name, self.card.name_diff, self.card.color, self.card.points, self.card.cost, self.card.feature, self.card.nr)
    
    def player_score(self):
        '''Shows score for player as (victory-points, sum of expeditions).'''
        return (self.data_player['points'], self.data_player['expeditions'])
    
    def player_statistics(self):
        '''Shows statistics for player.'''
        return self.data_player
    
    def draw_row(self, desc_str, data_str):
        '''Function draws a row with specific data.'''
        how_spaces = self.long - (len(desc_str) + len(str(data_str)) + self.long-(self.long-3))
        return '\n| ' + desc_str + ': ' + str(data_str) + ' '*(how_spaces) + '|'
    
    def draw_card(self):
        '''Function draws a card.'''
        descs = {self.card_description['color'][self.ch_lang]:self.card.color,
                 self.card_description['points'][self.ch_lang]:self.card.points,
                 self.card_description['cost'][self.ch_lang]:self.card.cost, 
                 self.card_description['feature'][self.ch_lang]:self.card.feature[0],
                 self.card_description['name'][self.ch_lang]:self.card.name_diff,
                 self.card_description['nr'][self.ch_lang]:self.card.nr
                }
        self.long = 32
        top_line = ' ' + '_'*self.long
        bottom_line = '|' + '_'*self.long + '|'
        mid = '|' + ' '*self.long + '|'
        mid_all = ''
        for key, val in descs.items():
            if val != None:
                mid_all += self.draw_row(key, val)
            elif val == None:
                mid_all += self.draw_row(key, '-')
            else:
                mid_all += mid
        full_picture = '\n'.join(map(str, (top_line, mid, mid_all.lstrip(), mid, mid, mid, mid, mid, mid, mid, bottom_line)))
        return full_picture      

In [35]:
class Game(object):
    def __init__(self, number_of_players):
        self.settings = Settings().settings
        self.lang = self.settings['choosen_language']
        self.yesno = self.settings['yes_no'][self.lang]
        self.players = [Player(gid) for gid in range(number_of_players)]
        self.names = {}
        self.deck = Deck()
        self.discarded = [] # list of discarded cards during game by players
        self.table = [] # cards list laying on table - first phase of game
        self.numbers_on_table = {} # cards' numbers laying on table {KEY:VAL -> Index:Number}
        self.card_nr = {} # catch {card_number:Card-object}
        self.str_nr = [str(idx) for idx in range(121)] # strings for int of cards' numbers - for compare
        self.taken = [] # cards' numbers taken by player in round and they are unavailavle
        
    def _allot_card(self, player):
        '''Function allot card choosen from deck for player.'''
        card = self.deck.choose_card_from_deck()
        player.pickup_card(card)
        
    def _card_from_table(self, player, idx):
        '''Function shows player's card laying on table - player owner.'''
        card = self.table[idx]
        player.pickup_card(card)
        
    def _determine_winner(self):
        '''Looking for a winner by comparing cards.'''
        ### !!!
        ### Will be change on comparing points and NO comparing sum of expeditions!
        ### The only one conditions with expeditions is collect MINIMUM 1 card of these type!!!
        won_set = max(self.players, key=lambda pid: pid.player_score())
        return won_set
        
    def _announce_winner(self, winner):
        '''Announce the winner - player with VP over 24 and MIN 1 expedition card.'''
        won_score = winner.player_score()
        print(self.settings['winner_info'][self.lang].format(winner.pid, won_score[0], won_score[1]))
        return winner

    def _meet_players(self):
        '''Function gets players' names and save to dictionary.'''
        for pid in range(len(self.players)):
            self.names[self.players[pid]] = input(self.settings['hello_name'][self.lang].format(pid))

    def _greetings(self):
        '''Says 'Hello' to players.'''
        print(self.settings['greetings'][self.lang])
        for name in sorted(self.names.values()):
            print(name, end=' ')

            ###
            ### to change
            ###
    def _cards_on_table(self, player):
        '''Function is dependent by choosing cards and showing table status.'''
        clear_output() # Using for cleaning IPython/Jupyter output
        self._allot_card(player) # Allot card from deck to player
        self.table.append(player.card) # Add to table cards choosen card
        self.card_nr[player.show_card()[-1]] = player.card # Add KEY=card_number and VALUE=Card object to dictionary
        print(self.settings['cards_on_table'][self.lang].format(len(self.table)))
        for idx_on_table in range(len(self.table)):
            self._card_from_table(player, idx_on_table) # 'Pick up' card from table, it will be use to draw card
            self.numbers_on_table[idx_on_table] = player.show_card()[-1] # Create dictionary with KEY=IDNDEX_FOR_CARD_ON_TABLE and VALUE=card_number
            print(player.draw_card()) # Drawing card
            
            #This part is dependent by 'logic'
            if player.show_card()[0] == 'ship': # If card is SHIP
                if player.show_card()[-2] != 0: # and IF Feature is not equal 0 - it will be changed on 9999 number or sth else
                    if player.player_statistics()['sabres']+7 >= player.show_card()[-2]: # If player has sum of sabres GE sabres on ship card
                        if input(self.settings['deflect_ship'][self.lang] + self.yesno).lower() in self.settings['yes_answers']: # and if answer is positive
                            self.table.remove(self.card_nr[player.show_card()[-1]]) # If player deflected ship, card go to trash
                            self.numbers_on_table_tmp = self.numbers_on_table.copy() # And number of card is not on table
                            for key, val in self.numbers_on_table_tmp.items(): # for every card on table
                                if val == player.show_card()[-1]: # if number equal player's 'catching' card
                                    self.numbers_on_table.pop(key) # card is remove prom table list
                                    
            if len(self.deck.expeditions) > 0: # if are any expeditions
                print(self.settings['are_expeditions'][self.lang]) # it shows
                
        while input(self.settings['next_card'][self.lang] + self.yesno).lower() in self.settings['yes_answers']:
            self._cards_on_table(player) # choose card again
        else:
            pass # in other case, finish function
                
    def _cards_from_hand(self, player):
        '''Function shows cards owned by player.'''
        print(self.settings['cards_have'][self.lang])
        print(player.cards_in_hand())
        for hand_card in player.cards_in_hand():
            player.pickup_card(hand_card)
            print(player.draw_card())
            
    def _clean_table(self):
        '''Cleaning table.'''
        self.discarded += self.table.copy()
        self.table = []
        self.numbers_on_table = {}
        self.card_nr = {}
        self.taken = []
        
    def _show_available_on_table(self):
        '''Shows cards' numbers laying on table which are available to take by player.'''
        print(self.settings['available_cards'][self.lang])
        for card_number in self.numbers_on_table.values():
             print(card_number, end=' ')
                
### CHANGE FOR LIKE ONE DEF!!!

             ###
            ### to change
            ###       
    def _card_selection(self, player):
        '''Player selections which cards from table want to buy or sell.'''
        statistics = player.player_statistics()
        cards_amount = 1 + statistics['extra_card'] # how many cards player can buy/sell
        self._show_available_on_table() # show availa ble cards for buy or sell
        print('\n')
        print(self.settings['buy_sell'][self.lang])
        while cards_amount > 0 and len(self.table) > 0: # while player can get more than 0 cards and on the table is more than 0 cards
            print(self.settings['can_take'][self.lang].format(cards_amount))
            choice = input(self.settings['nr_or_enter'][self.lang])
            if choice in self.str_nr: # if card number is in numbers on the table
                if int(choice) in self.numbers_on_table.values() and int(choice) not in self.taken: # up and number not in taken cards
                    for key, val in self.numbers_on_table.items(): # for every card on table
                        if int(choice) == self.numbers_on_table[key]: #if number equal number from numbers on table
                            player.pickup_card(self.card_nr[int(choice)]) # player get card BUT...
                            if player.show_card()[0] == 'ship': # ...if card is ship
                                statistics['gold'] += player.show_card()[4] # player got GOLD
                            else:
                                how_much = player.show_card()[-3] - statistics['pay_less'] # player GOLD is summing, if players has benefits to pay less for cards
                                if how_much >= statistics['gold']: # and player has gold-1 for choosen card
                                    statistics['gold'] -= how_much # gold is going to somewhere (minus)
                            self._card_from_table(player, key) # AND card from table is assigning to player
                            player.take_card() # above
                            print(self.settings['taken_nr'][self.lang].format(choice))
                            self.taken.append(self.numbers_on_table[key]) # add taken card to list of taken cards
                            self.table.remove(self.card_nr[int(choice)]) # and card number is removing from table
                            cards_amount -= 1 # on the end, amount of available cards is reduced at 1
                else:
                    print(self.settings['bad_value'][self.lang])
            else:
                print(self.settings['finish_trade'][self.lang])
                cards_amount = 0
    
    def start_game(self):
        self.deck.reset_choosen_and_expeditions() # Reset deck on begining of every game.
        self._meet_players()
        self._greetings()
        for pid in self.players:
            print(self.settings['tour_of'][self.lang].format(self.names[pid]))
            self._cards_on_table(pid)
            if len(self.table) > 0:
                print('Zakonczono faze dobierania kart. Rozpoczyna sie faza handlu')
                self._card_selection(pid)
            else:
                print('Zakonczono faze dobierania kart. Ze wzgledu na brak kart na stole - faza handlu zostaje pominieta')
            self._clean_table()
            print('---karty gracza {} ---'.format(self.names[pid]))
            self._cards_from_hand(pid)
            print(pid.player_statistics())
            print('karta_nr - na stole')
            print(self.card_nr)
            print('karty na stole')
            print(self.numbers_on_table)
            print('karty na stole - self.table')
            print(self.table)
            print('karty w hasioku')
            print(self.discarded)
            

#         winner = self._determine_winner()
#         return self._announce_winner(winner)

In [36]:
r = Game(number_of_players=1)
r.start_game()

Karty na stole: 1

 ________________________________
|                                |
| Koszt: 4                       |
| Nazwa: osadnik                 |
| Punkty: 1                      |
| Kolor: -                       |
| Cecha: houses                  |
| Numer: 15                      |
|                                |
|                                |
|                                |
|                                |
|                                |
|                                |
|                                |
|________________________________|
Losować następną kartę? T/[n]
Zakonczono faze dobierania kart. Rozpoczyna sie faza handlu
Numery dostępnych kart:
15 Wpisz numer karty którą chcesz sprzedać lub kupić. Jeśli chcesz zakończyę fazę handlu - wciśnij ENTER
Możesz kupić lub sprzedać jeszcze 1 kart.
Wpisz numer lub naciśnij ENTER: 
Zakończono fazę handlu.
---karty gracza jan ---
Karty, ktore posiadasz.
[]
{'sabres': 0, 'extra_money_fivecards': 0, 'crosses': 

In [63]:
import json

settings = {'language':{'eng':0, 'pol':1},
            'choosen_language':1,
            'hello_name':('Hello Player{}. Type your name: ', 'Witaj Gracz{}. Podaj swoje imię: '),
            'yes_no':(' Y/[n]', ' T/[n]'),
            'yes_answers':('t', 'tak', 'y', 'yes'),
            'no_answers':('n', 'nie', 'no'),
            'winner_info':('Player {} has won, had got {} Victory Points and {} expeditions card(s).', 'Wygrał gracz o ID {} mając {} Punktów Zwycięstwa i {} kart(ę) ekspedycji.\n'),
            'greetings':('I have pleasure invite you to game:', 'Milo mi was zaprosic do gry:'),
            'cards_have':('Cards which you have.','Karty, ktore posiadasz.'),
            'available_cards':('Available cards\' numbers:', 'Numery dostępnych kart:'),
            'cards_on_table':('Cards on table: {}\n','Karty na stole: {}\n'),
            'deflect_ship':('Do you wanna deflect a ship?', 'Czy chcesz odeprzec statek?'),
            'are_expeditions':('Expeditions are available.', 'Są dostępne ekspedycje.'),
            'next_card':('Do you wanna take next card from deck?', 'Losować następną kartę?'),
            'buy_sell':('Type number of card which you want to buy or sell. If you want to finish trade phase - press ENTER.', 'Wpisz numer karty którą chcesz sprzedać lub kupić. Jeśli chcesz zakończyę fazę handlu - wciśnij ENTER'),
            'can_take':('You can buy or sell even {} cards.', 'Możesz kupić lub sprzedać jeszcze {} kart.'),
            'nr_or_enter':('Type number or press ENTER: ', 'Wpisz numer lub naciśnij ENTER: '),
            'bad_value':('Bad value. Please try again.', 'Zła wartość. Spróbuj jeszcze raz.'),
            'taken_nr':('Card number {} has taken.', 'Wzięto kartę numer {}.'),
            'finish_trade':('Trade phase has finished.', 'Zakończono fazę handlu.'),
            'tour_of':('Tour of player {}.', 'Kolejka gracza {}.')
            }

card_description = {'color':('Color', 'Kolor'),
                    'points':('Points', 'Punkty'),
                    'cost':('Cost', 'Koszt'),
                    'feature':('Feature', 'Cecha'),
                    'name':('Name', 'Nazwa'),
                    'nr':('Number', 'Numer'),
                    'anchors':('anchors', 'kotwice'),
                    'anchors_houses':('anchors and houses', 'kotwice i domy'),
                    'black_more':('extra gold coin when buy black ship', 'dodatkowa moneta przy zakupie czarnego statku'),
                    'blue_more':('extra gold coin when buy blue ship', 'dodatkowa moneta przy zakupie niebieskiego statku'),
                    'crosses':('crosses', 'krzyże'),
                    'crosses_houses':('crosses and houses', 'krzyże i domy'),
                    'extra_card':('extra card from table', 'dodatkowa karta ze stołu'),
                    'extra_money_fivecards':('5 coins when lay 5 or more cards', '5 monet za wyłożenie minimum 5 kart'),
                    'extra_money_notrade':('extra coin when you cannot trade', 'dodatkowa moneta za brak możliwości handlu'),
                    'green_more':('extra gold coin when buy green ship', 'dodatkowa moneta przy zakupie zielonego statku'),
                    'houses':('houses', 'domy'),
                    'houses_anchors_crosses':('houses, anchors and crosses', 'domy, kotwice i krzyże'),
                    'jokers':('jokers', 'dżokery'),
                    'pay_less':('1 coin less when you buy any card', '1 moneta mniej przy zakupie dowolnej karty'),
                    'red_more':('extra gold coin when buy red ship', 'dodatkowa moneta przy zakupie czerwonego statku'),
                    'sabres':('sabres', 'szable'),
                    'tax_points':('tax: minimum Victory Points +1 coin; if player has 12+ coins - tax take 50%', 'podatek - minimum Punktów Zwycięstwa +1 moneta; jeśli gracz ma 12+ monet - podatek zabiera 50%'),
                    'tax_sabres':('tax - maximum sabres +1 coin; if player has 12+ coins - tax take 50%', 'podatek - maximum szabli + 1 moneta; jeśli gracz ma 12+ monet - podatek zabiera 50%'),
                    'yellow_more':('extra gold coin when buy yellow ship', 'dodatkowa moneta przy zakupie żółtego statku')
                    }

print(settings)
with open('card_description.ini', 'w') as f:
    json.dump(card_description, f)
    
with open('game_settings.ini', 'w') as f:
    json.dump(settings, f)

{'are_expeditions': ('Expeditions are available.', 'Są dostępne ekspedycje.'), 'can_take': ('You can buy or sell even {} cards.', 'Możesz kupić lub sprzedać jeszcze {} kart.'), 'next_card': ('Do you wanna take next card from deck?', 'Losować następną kartę?'), 'bad_value': ('Bad value. Please try again.', 'Zła wartość. Spróbuj jeszcze raz.'), 'winner_info': ('Player {} has won, had got {} Victory Points and {} expeditions card(s).', 'Wygrał gracz o ID {} mając {} Punktów Zwycięstwa i {} kart(ę) ekspedycji.\n'), 'yes_no': (' Y/[n]', ' T/[n]'), 'taken_nr': ('Card number {} has taken.', 'Wzięto kartę numer {}.'), 'nr_or_enter': ('Type number or press ENTER: ', 'Wpisz numer lub naciśnij ENTER: '), 'language': {'pol': 1, 'eng': 0}, 'cards_on_table': ('Cards on table: {}\n', 'Karty na stole: {}\n'), 'hello_name': ('Hello Player{}. Type your name: ', 'Witaj Gracz{}. Podaj swoje imię: '), 'greetings': ('I have pleasure invite you to game:', 'Milo mi was zaprosic do gry:'), 'tour_of': ('Tour of

In [61]:
print('\%')

\%


In [40]:
with open('pr_deck.txt', 'r') as f: talia=json.load(f)

In [59]:
vals=set()
for idx in range(len(talia)):
    vals.add(talia[idx][4][0])
vals

{'anchors',
 'anchors_houses',
 'black_more',
 'blue_more',
 'crosses',
 'crosses_houses',
 'extra_card',
 'extra_money_fivecards',
 'extra_money_notrade',
 'green_more',
 'houses',
 'houses_anchors_crosses',
 'jokers',
 'pay_less',
 'red_more',
 'sabres',
 'tax_points',
 'tax_sabres',
 'yellow_more'}

deck3 = {}
for idx in deck2:
    deck3[idx[-1]] = idx

with open('deck_portroyal.txt', 'w') as f:
    json.dump(deck2, f)

with open('deck_portroyal.txt', 'r') as f:
    deck4 = json.load(f)